In [1]:
import json
import torch.optim as optim
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import random
import torch
# 一些常量
# On windows
# dir_train = 'D:/Github/candidate_answer/data/json_train_expt_stop'
# dir_test = 'D:/Github/candidate_answer/data/json_test_expt_stop'
# dir_embedding = 'D:/nlp_data/sogou_100_nobinary'
# On ubuntu
dir_train = 'data/json_train_expt_stop2'
dir_test = 'data/json_test_expt_stop'
dir_embedding = '/home/tuomx/nlp_data/sogou_100_nobinary'

embedding_size = 100
max_question_words = 23 # 问题最大词数，下同理
max_right_answer_words = 351
max_wrong_answer_words = 824
kernel_size = (3, embedding_size) # 卷积核的size
out_channels = 300 # 输出通道数
hidden_out = 400 # 隐藏层输出单元数
batch_size = 16

In [2]:
# 加载词向量
embedding = {}
f = open(dir_embedding,"r",encoding='utf-8')
line = f.readline()
line_num = 0
print("loading enmbedding...")
while line:
    try:
        content = line.strip(' \n').split(' ')
        assert len(content) == embedding_size + 1
        embedding[content[0]] = np.array([float(i) for i in content[1:]])
        line = f.readline()
        line_num+=1
#         print(line_num)
    except:
        print(content)
        break
print("finish loading")


loading enmbedding...
finish loading


In [7]:
# 创建一个CNN\

# caculate hinge_loss
def hinge_loss(s1,s2,t0,batch_size):
    # print(s1.size(), s2.size())
    loss = Variable(torch.Tensor(1))
    loss.data[0] = 0.0
    for i in range(batch_size):
        if (t0 - s1[i] + s2[i]).data[0] > 0:
            loss += t0 - s1[i] + s2[i]
        else:
            print("pos:%f,neg:%f" % (s1[i].data[0], s2[i].data[0]))
    return loss
    
def get_simple_score(self,question_variable, pos_relation_variable_l, pos_word_variable_l, neg_relation_variable_l, neg_word_variable_l, hn_hidden,cn_hidden):
        '''计算一个问题的分数，正例与负例，用max——margin计算
        '''
        sum_score = 0.0

        qr_output = self.qr_model(question_variable,hn_hidden,cn_hidden,self.padding_size)
        pos_score_list = []
        neg_score_list = []
        for pi in range(len(pos_relation_variable_l)):
            # self.optimizer.zero_grad()
            pos_ar_output = self.ar_model(pos_relation_variable_l[pi],pos_word_variable_l[pi],hn_hidden,cn_hidden,20)
            pos_score = F.cosine_similarity(qr_output,pos_ar_output)
            pos_score_list.append(pos_score)

        for ni in range(len(neg_relation_variable_l)):
            # self.optimizer.zero_grad()
            neg_ar_output = self.ar_model(neg_relation_variable_l[ni],neg_word_variable_l[ni],hn_hidden,cn_hidden,20)
            neg_score = F.cosine_similarity(qr_output,neg_ar_output)
            neg_score_list.append(neg_score)

        for pos_score in pos_score_list:
            for neg_score in neg_score_list:
                score = self.max_margin_number - pos_score + neg_score
                if score.data[0] <0:
                    score = Variable(torch.FloatTensor(1).fill_(0.0))
                sum_score += score

        return sum_score

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 300, (3,100)) # 保证输出列向量在高度上与X相同
        self.conv2 = nn.Conv2d(1, 300, (3,100))
        self.conv3 = nn.Conv2d(1, 300, (3,100))

        self.pool1 = nn.MaxPool2d(1, max_wrong_answer_words) # 输出是out_channels*1维向量
        self.pool2 = nn.MaxPool2d(1, max_question_words)
        self.pool3 = nn.MaxPool2d(1, max_right_answer_words)

        self.fc1 = nn.Linear(out_channels, hidden_out)
        self.fc2 = nn.Linear(out_channels, hidden_out)
        self.fc3 = nn.Linear(out_channels, hidden_out)


    def forward(self, x1, x2, x3, batch_size):
        # x1/x2/x3 分别表示错误答案，问题，正确答案
#         x1 = self.pool1(F.tanh(self.conv1(x1)))
#         x2 = self.pool2(F.tanh(self.conv2(x2)))
#         x3 = self.pool3(F.tanh(self.conv3(x3)))
        # print("in forward:")
        # print("X:",x1.size(),x2.size(),x3.size())
        x1 = F.tanh(self.conv1(x1))
        x2 = F.tanh(self.conv2(x2))
        x3 = F.tanh(self.conv3(x3))
        #print("conv1:",x1.size(),x2.size(),x3.size())
        
        x1 = self.pool1(x1)
        x2 = self.pool2(x2)
        x3 = self.pool3(x3)
        #print("pool:",x1.size(),x2.size(),x3.size())
        
        x1 = F.tanh(x1)
        x2 = F.tanh(x2)
        x3 = F.tanh(x3)

        neg_cosine = F.cosine_similarity(x1,x2)
        pos_cosine = F.cosine_similarity(x2,x3)
        #print(neg_cosine, pos_cosine)

        return hinge_loss(pos_cosine, neg_cosine, 2, batch_size), pos_cosine, neg_cosine
net = Net()

In [8]:
# 开始训练
def get_sentence_embedding(s,out_size):
    arr = []
    for word in s:
        if word in embedding:
            arr.append(embedding[word])
        else:
            arr.append([random.uniform(-1,1) for i in range(embedding_size)])
    if len(arr) < out_size: # 补零
        append_arr = [0.0 for i in range(embedding_size)]
        for j in range(out_size - len(arr)):
            arr.append(append_arr)
    return [arr]

optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
f = open(dir_train,'r',encoding='utf-8')
data = json.loads(f.read()) # 8768 quesions(except 4 questions which don't have the right answer)
count_step = 0
for epoch in range(2):  # loop over the dataset multiple times
    f = open(dir_train)
    running_loss = 0.0
    batch = [[] for i in range(3)]
    for id in data:
        # stop early
#         if count_step == 5000:
#             print("finish training")
#             break
        
        # get the inputs
        question_ebd = get_sentence_embedding(data[id]['question'], max_question_words)
        right_answer_ebd = get_sentence_embedding(data[id]['right_answer'][0], max_right_answer_words)
        for wrong_answer in data[id]['wrong_answer']:
            wrong_answer_ebd = get_sentence_embedding(wrong_answer, max_wrong_answer_words)
            batch[0].append(wrong_answer_ebd)
            batch[1].append(question_ebd)
            batch[2].append(right_answer_ebd)
            if len(batch[0]) == batch_size:
                # wrap them in Variable
                # assert(batch[0])
        
                x1 = Variable(torch.from_numpy(np.array(batch[0])).float())
                x2 = Variable(torch.from_numpy(np.array(batch[1])).float())
                x3 = Variable(torch.from_numpy(np.array(batch[2])).float())
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward + backward + optimize
                loss,pos_cosine,neg_cosine = net(x1, x2, x3, batch_size)
                if loss.data[0] != 0.0:
                    loss.backward()
                    optimizer.step()

                    # print statistics
                    count_step += 1
                    running_loss += loss.data[0]
    #                print("Epoch:%d Step:%d: Loss:%f" % (epoch, count_step, loss.data[0]))
                    if count_step % 200 == 199:    # print every 2000 mini-batches
                        print('[%d, %5d] loss: %.3f' %
                              (epoch + 1, count_step + 1, running_loss / 200))
                        running_loss = 0.0
                # clear batch
                batch = [[] for i in range(3)]

print('Finished Training')

[1,   200] loss: 26.263
[1,   400] loss: 17.197
[1,   600] loss: 11.845
[1,   800] loss: 7.654
[1,  1000] loss: 6.437
[1,  1200] loss: 4.667
[1,  1400] loss: 3.926
[1,  1600] loss: 2.788
[1,  1800] loss: 2.745
[1,  2000] loss: 2.307
[1,  2200] loss: 2.253
[1,  2400] loss: 2.317
[1,  2600] loss: 2.040
[1,  2800] loss: 1.135
[1,  3000] loss: 1.475
[1,  3200] loss: 1.167


KeyboardInterrupt: 

In [22]:
# test
print('start test...')
f = open(dir_test,'r',encoding='utf-8')
test_data = json.loads(f.read())

MRR = 0
for id in test_data:
    print(id)
    rank = 1
    # get the inputs
    question_ebd = get_sentence_embedding(test_data[id]['question'], max_question_words)
    right_answer_ebd = get_sentence_embedding(test_data[id]['right_answer'][0], max_right_answer_words)
    for wrong_answer in data[id]['wrong_answer']:
        batch = [[] for i in range(3)]
        wrong_answer_ebd = get_sentence_embedding(wrong_answer, max_wrong_answer_words)
        batch[0].append(wrong_answer_ebd)
        batch[1].append(question_ebd)
        batch[2].append(right_answer_ebd)

        x1 = Variable(torch.from_numpy(np.array(batch[0])).float())
        x2 = Variable(torch.from_numpy(np.array(batch[1])).float())
        x3 = Variable(torch.from_numpy(np.array(batch[2])).float())
        # print(x1.size(),x2.size(),x3.size())
        loss,pos_cosine,neg_cosine = net(x1,x2,x3,1)
        print(pos_cosine.data[0],neg_cosine.data[0])
        if (pos_cosine.data[0] < neg_cosine.data[0]):
            rank += 1
    MRR += 1 / rank
MRR /= len(test_data)

start test...
1
0.9912660121917725 -0.8700141310691833
0.9912660121917725 -0.9835314154624939
0.9912660121917725 -0.5564292669296265
0.9912660121917725 -0.8491112589836121
0.9912660121917725 -0.9828693866729736
0.9912660121917725 -0.9894813299179077
0.9912660121917725 -0.7649186849594116
0.9912660121917725 -0.9889241456985474
0.9912660121917725 -0.9529318809509277
0.9912660121917725 -0.9835314154624939
0.9912660121917725 -0.9828693866729736
0.9912660121917725 -0.9828693866729736
0.9912660121917725 -0.9816619753837585
0.9912660121917725 -0.9630176424980164
0.9912660121917725 -0.8149563670158386
0.9912660121917725 -0.9895715117454529
0.9912660121917725 -0.9918650984764099
0.9912660121917725 -0.9902403354644775
0.9912660121917725 -0.9906905293464661
0.9912660121917725 -0.9879118204116821
0.9912660121917725 -0.991284191608429
0.9912660121917725 -0.9871680736541748
0.9912660121917725 -0.9912195801734924
0.9912660121917725 -0.9903814792633057
0.9912660121917725 -0.7644052505493164
0.99126601

0.9924654364585876 -0.9920088648796082
0.9924654364585876 -0.991895854473114
0.9924654364585876 -0.9908676743507385
0.9924654364585876 -0.9918937683105469
0.9924654364585876 -0.9917209148406982
0.9924654364585876 -0.9880845546722412
0.9924654364585876 -0.9900782704353333
0.9924654364585876 -0.9904935956001282
0.9924654364585876 -0.9906586408615112
0.9924654364585876 -0.9916619062423706
0.9924654364585876 -0.9924728274345398
11
0.9886641502380371 -0.9907407760620117
0.9886641502380371 -0.990084707736969
0.9886641502380371 -0.9900126457214355
0.9886641502380371 -0.9882339239120483
0.9886641502380371 -0.9880317449569702
0.9886641502380371 -0.9898210167884827
0.9886641502380371 -0.9896901249885559
0.9886641502380371 -0.9900579452514648
0.9886641502380371 -0.9909811615943909
0.9886641502380371 -0.9896582365036011
0.9886641502380371 -0.9890344142913818
0.9886641502380371 -0.9828059077262878
0.9886641502380371 -0.9786614775657654
0.9886641502380371 -0.9880900382995605
0.9886641502380371 -0.98

0.983702540397644 -0.9932316541671753
0.983702540397644 -0.9940422773361206
0.983702540397644 -0.9875403642654419
0.983702540397644 -0.9473713040351868
0.983702540397644 -0.9945834279060364
0.983702540397644 -0.994113564491272
0.983702540397644 -0.9939176440238953
0.983702540397644 -0.993331789970398
0.983702540397644 -0.9937563538551331
0.983702540397644 -0.9938570261001587
0.983702540397644 -0.9945922493934631
0.983702540397644 -0.990704357624054
0.983702540397644 -0.9940595030784607
0.983702540397644 -0.9941186904907227
0.983702540397644 -0.9944730401039124
0.983702540397644 -0.9946750402450562
0.983702540397644 -0.9941416382789612
0.983702540397644 -0.9932615756988525
0.983702540397644 -0.9938239455223083
19
0.9850583672523499 -0.9967040419578552
0.9850583672523499 -0.997551679611206
0.9850583672523499 -0.9959759712219238
0.9850583672523499 -0.997551679611206
20
0.969774067401886 -0.9867779016494751
0.969774067401886 -0.9852592349052429
0.969774067401886 -0.9845722317695618
0.96977

0.9935735464096069 -0.9970976114273071
0.9935735464096069 -0.9981445074081421
0.9935735464096069 -0.9974649548530579
0.9935735464096069 -0.9970659613609314
0.9935735464096069 -0.9958678483963013
28
0.9937496185302734 -0.9962992072105408
0.9937496185302734 -0.9962992072105408
0.9937496185302734 -0.993080735206604
0.9937496185302734 -0.9966937303543091
0.9937496185302734 -0.997065544128418
0.9937496185302734 -0.9946370124816895
0.9937496185302734 -0.990530788898468
0.9937496185302734 -0.9864944815635681
0.9937496185302734 -0.9965060949325562
0.9937496185302734 -0.9964641332626343
0.9937496185302734 -0.9956629276275635
0.9937496185302734 -0.9969009160995483
0.9937496185302734 -0.9959214925765991
0.9937496185302734 -0.9970194101333618
0.9937496185302734 -0.9969576001167297
0.9937496185302734 -0.9972118139266968
0.9937496185302734 -0.9928238987922668
0.9937496185302734 -0.9928238987922668
0.9937496185302734 -0.9956550598144531
0.9937496185302734 -0.9960326552391052
0.9937496185302734 -0.975

0.9979679584503174 -0.9833349585533142
0.9979679584503174 -0.9977452158927917
0.9979679584503174 -0.9816003441810608
0.9979679584503174 -0.997590959072113
0.9979679584503174 -0.9872000217437744
0.9979679584503174 -0.977111279964447
0.9979679584503174 -0.9967061877250671
0.9979679584503174 -0.9953579306602478
0.9979679584503174 -0.9939836263656616
0.9979679584503174 -0.9984043836593628
0.9979679584503174 -0.9987773895263672
0.9979679584503174 -0.9988939166069031
0.9979679584503174 -0.9506788849830627
0.9979679584503174 -0.9945183396339417
0.9979679584503174 -0.9980501532554626
0.9979679584503174 -0.9986134171485901
0.9979679584503174 -0.9981628060340881
0.9979679584503174 -0.9982842206954956
0.9979679584503174 -0.9976904988288879
0.9979679584503174 -0.9963361620903015
0.9979679584503174 -0.9981663227081299
36
0.9930866360664368 -0.9943665862083435
0.9930866360664368 -0.9956261515617371
0.9930866360664368 -0.9845607280731201
0.9930866360664368 -0.9953179359436035
0.9930866360664368 -0.99

...

0.9809706807136536 -0.9517349600791931
0.9809706807136536 -0.9895601272583008
0.9809706807136536 -0.9875775575637817
0.9809706807136536 -0.989396870136261
0.9809706807136536 -0.9782233238220215
0.9809706807136536 -0.9889837503433228
0.9809706807136536 -0.9856091141700745
0.9809706807136536 -0.9842482805252075
0.9809706807136536 -0.9586989283561707
77
0.9948975443840027 -0.9914368987083435
0.9948975443840027 -0.9917280673980713
0.9948975443840027 -0.9701641798019409
0.9948975443840027 -0.9975782632827759
0.9948975443840027 -0.9868264198303223
0.9948975443840027 -0.9983298182487488
0.9948975443840027 -0.9649333357810974
0.9948975443840027 -0.996813178062439
0.9948975443840027 -0.9949707984924316
0.9948975443840027 -0.9947924017906189
0.9948975443840027 -0.9724047183990479
0.9948975443840027 -0.9918305277824402
0.9948975443840027 -0.9981104135513306
0.9948975443840027 -0.9841543436050415
0.9948975443840027 -0.9924002885818481
0.9948975443840027 -0.9726690649986267
0.9948975443840027 -0.98

0.9927244186401367 -0.9972447156906128
0.9927244186401367 -0.9940665364265442
0.9927244186401367 -0.9940803647041321
0.9927244186401367 -0.9968701004981995
0.9927244186401367 -0.9933967590332031
0.9927244186401367 -0.9964107871055603
0.9927244186401367 -0.9872843623161316
0.9927244186401367 -0.995620608329773
0.9927244186401367 -0.9922620058059692
0.9927244186401367 -0.9723432660102844
0.9927244186401367 -0.9925049543380737
0.9927244186401367 -0.9936608076095581
0.9927244186401367 -0.9871907830238342
0.9927244186401367 -0.9922923445701599
0.9927244186401367 -0.9918680191040039
0.9927244186401367 -0.9976221323013306
0.9927244186401367 -0.9952540993690491
0.9927244186401367 -0.9975099563598633
0.9927244186401367 -0.9961467981338501
85
0.9952812790870667 -0.991112232208252
0.9952812790870667 -0.9978415966033936
0.9952812790870667 -0.9967920780181885
0.9952812790870667 -0.995523989200592
0.9952812790870667 -0.991112232208252
0.9952812790870667 -0.9967920780181885
0.9952812790870667 -0.9955

0.9952456951141357 -0.9893257021903992
0.9952456951141357 -0.9928377866744995
0.9952456951141357 -0.9834503531455994
0.9952456951141357 -0.9923438429832458
0.9952456951141357 -0.9901407957077026
0.9952456951141357 -0.9926010966300964
0.9952456951141357 -0.9708549976348877
0.9952456951141357 -0.9943339228630066
0.9952456951141357 -0.9762884378433228
0.9952456951141357 -0.9927664995193481
0.9952456951141357 -0.9912939667701721
0.9952456951141357 -0.9951459169387817
0.9952456951141357 -0.991572916507721
0.9952456951141357 -0.9856371879577637
0.9952456951141357 -0.9927664995193481
0.9952456951141357 -0.9948253631591797
0.9952456951141357 -0.9893415570259094
0.9952456951141357 -0.9920289516448975
0.9952456951141357 -0.990791380405426
0.9952456951141357 -0.9951459169387817
0.9952456951141357 -0.991572916507721
0.9952456951141357 -0.9856371879577637
0.9952456951141357 -0.9927664995193481
0.9952456951141357 -0.9948253631591797
0.9952456951141357 -0.9708549976348877
0.9952456951141357 -0.994333

0.9966626763343811 -0.9980586171150208
0.9966626763343811 -0.9900569915771484
0.9966626763343811 -0.9955506324768066
0.9966626763343811 -0.9980586171150208
0.9966626763343811 -0.9920122027397156
0.9966626763343811 -0.9980648159980774


KeyboardInterrupt: 

In [20]:
MRR / 454

0.9977973568281938